In [ ]:
%%capture
!pip install duckdb

In [ ]:
#Librerias básicas
#================================================================================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import duckdb
import glob

# Configuración para machine learning
# ================================================================================
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from pandas.core.frame import DataFrame
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import multiprocessing

import cv2

# Activar SQL en Python
# ================================================================================
con = duckdb.connect(database=':memory:')

In [ ]:
url_lcc = "/kaggle/input/curvasluz/Curvas_de_luz_cor/"
url    = "/kaggle/input/carac-stars/"

## Creación de datos de entrenamiento, validación y prueba

In [ ]:
Carac_Stars = pd.read_csv(url+"Carac_Stars.csv")
Carac_Stars

In [ ]:
arch = os.listdir(url_lcc) 
arch.sort()
Stars_list = [star[3:-4] for star in arch]

s_flux  = []
s_err   = []
Id_real = []
minn=[]
for Str in Stars_list:
    serie = pd.read_csv(url_lcc+'LC_'+Str+'.csv').set_index('time')
    s_flux.append(np.array(serie['flux'])[0:1284])
    s_err.append(np.array(serie['flux_err'])[0:1284])

In [ ]:
Stars = pd.DataFrame()
Stars["ID"] = Stars_list
Stars

In [ ]:
Clas_Stars = con.execute("""select S.ID, C.Metalicidad, case when C.Clasificacion == 'CANDIDATE' then 1 
                                              else 0             
                                         end as ClasBin
                            from Carac_Stars C inner join Stars S on (S.ID = C.Kepler_ID)
                            order by S.ID""").df()
Clas_Stars

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(s_flux[198])
#plt.title(f'Curva de luz de la estrella {Stars[190]}') 
plt.ylabel('Flujo')
plt.xlabel('Tiempo')

plt.subplot(1,2,2)

plt.plot(s_err[198])
#plt.title(f'Error de la curva de luz de la estrella {Stars[190]}') 
plt.ylabel('Error')
plt.xlabel('Tiempo')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(s_flux[6480])
#plt.title(f'Curva de luz de la estrella {Stars[6481]}') 
plt.ylabel('Flujo')
plt.xlabel('Tiempo')

plt.subplot(1,2,2)

plt.plot(s_err[6480])
#plt.title(f'Error de la curva de luz de la estrella {Stars[6481]}') 
plt.ylabel('Error')
plt.xlabel('Tiempo')

plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(s_flux), 
                                                    np.array(Clas_Stars["ClasBin"]),
                                                    test_size=0.3, random_state=13)

X_val,  X_test,  y_val,  y_test  = train_test_split(X_test, 
                                                    y_test,
                                                    test_size=0.333, random_state=13)
for i in range(len(X_train)):
    mean_i = np.mean(X_train[i])
    std_i  = np.std(X_train[i])
    X_train[i] = (X_train[i] - mean_i)/std_i
    
for i in range(len(X_val)):
    mean_i = np.mean(X_val[i])
    std_i  = np.std(X_val[i])
    X_val[i] = (X_val[i] - mean_i)/std_i

X_train = X_train.reshape((len(X_train), 1284, 1))
X_test  = X_test.reshape((len(X_test), 1284, 1))
X_val   = X_val.reshape((len(X_val), 1284, 1))

print("Tamaño de entrenamiento:",X_train.shape,
      "\nTamaño de validacion:",X_val.shape,
      "\nTamaño de prueba:", X_test.shape,
      "\nDatos totales:",len(X_train)+len(X_val)+len(X_test))

In [ ]:
len(y_train[y_train==1])

In [ ]:
len(y_train[y_train==0])

## Red Convolucional (CNN)

In [ ]:
from tensorflow.keras.layers import Conv1DTranspose, UpSampling1D, Reshape, Dropout

In [ ]:
### Se regula con Dropout
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='leaky_relu',padding="same", input_shape=(1284,1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=128, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=256, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=512, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(512, activation='leaky_relu'),
    Dropout(0.3),
    Dense(256, activation='leaky_relu'),
    Dropout(0.3),
    Dense(128, activation = 'leaky_relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
##### Modelo con Regularización L2
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='leaky_relu',padding="same", input_shape=(1284,1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=128, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=256, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=512, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(512, activation='leaky_relu', kernel_regularizer=l2(0.01)),
    Dense(256, activation='leaky_relu', kernel_regularizer=l2(0.01)),
    Dense(128, activation = 'leaky_relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='sigmoid')
])

In [ ]:
#CallBack para detener entrenamiento
class MNIST_Callback(tf.keras.callbacks.Callback):
    
    # método dentro de la clase myCallback, heredada de la clase Callback de keras
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.95):
            print("\nSe alcanzó un 98.5% de precisión en la validación! Cancelando Entrenamiento...")
            self.model.stop_training = True
        elif(logs.get('accuracy')>0.95):
            rint("\nSe alcanzó un 98.5% de precisión en el entrenaminto! Cancelando Entrenamiento...")
            self.model.stop_training = True


# crea una instancia de clase
accu_callback = MNIST_Callback()

In [ ]:
#Compilador del modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data = (X_val,y_val),
                    batch_size=256,
                    callbacks=[accu_callback])

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['val_loss'], label='val_accuracy')
plt.plot(history.history['loss'], label='accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle("Arquitectura con regularización L2", fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['val_loss'], label='val_accuracy')
plt.plot(history.history['loss'], label='accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.suptitle("Arquitectura con tasa de aprendizaje de 0.001", fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['val_loss'], label='val_accuracy')
plt.plot(history.history['loss'], label='accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.suptitle("Arquitectura con tasa de aprendizaje de 0.001", fontsize=20)
plt.show()

## Autoencoder para realizar la clasificación

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.layers import Conv1DTranspose, UpSampling1D, Reshape, Conv1D,MaxPooling1D,  Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:

input_series = Input(shape=(1284, 1))

#Encoder
encoded = Dense(512, activation='tanh')(input_series)
encoded = Dropout(0.5)(encoded)
encoded = Dense(256, activation='leaky_relu')(encoded)
encoded = Dropout(0.5)(encoded)
encoded = Dense(128, activation='leaky_relu')(encoded)
encoded = Dropout(0.5)(encoded)
#encoded = Dense(16, activation='leaky_relu')(encoded)
#encoded = Dropout(0.5)(encoded)

# Cuello de botella
bottleneck = Dense(1, activation='leaky_relu')(encoded)

#Decoder
#decoded = Dense(16, activation='leaky_relu')(bottleneck)
#decoded = Dropout(0.5)(decoded)
decoded = Dense(128, activation='leaky_relu')(bottleneck)
decoded = Dropout(0.5)(decoded)
decoded = Dense(256, activation='leaky_relu')(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(512, activation='tanh')(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(1, activation='leaky_relu')(decoded)


autoencoder = Model(input_series, decoded)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
autoencoder.compile(optimizer=optimizer,
              loss='mse')

autoencoder.summary()

In [ ]:
history = autoencoder.fit(X_train[y_train==0], X_train[y_train==0], 
                    epochs=20, 
                    validation_data = (X_val,X_val),
                    batch_size=256)

In [ ]:
plt.plot(history.history['loss'],'ro-', label='loss')
plt.plot(history.history['val_loss'],'bo-', label='val_loss')
plt.xlabel('Epochs')
plt.legend()

In [ ]:
encoder_model = Model(input_series, bottleneck)

#features_train = encoder_model.predict(X_train)
features_test = encoder_model.predict(X_test)
#features_val = encoder_model.predict(X_val)


In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.title("Curva Orginal")
plt.plot(X_train[3])

plt.subplot(1,2,2)
plt.title("Curva reconstruida")
plt.plot(features_train[3])

#plt.subplot(2,2,3)
#plt.plot(features_train[2])
#plt.plot(X_train[2])

#plt.subplot(2,2,4)
#plt.plot(features_train[3])
#plt.plot(X_train[3])

plt.savefig('Curvas3.png')
plt.show()

In [ ]:
ecm = np.mean(np.power(X_test-features_test,2), axis=1)

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, umbral = precision_recall_curve(y_test, ecm)

plt.plot(umbral, precision[1:], label="Precision",linewidth=5)
plt.plot(umbral, recall[1:], label="Recall",linewidth=5)
plt.title('Precision y Recall para diferentes umbrales')
plt.xlabel('Umbral')
plt.ylabel('Precision/Recall')
plt.legend()
plt.savefig("PrecisionRecall")
plt.show()

In [ ]:
umbral_fijo = 1e8
y_pred = [1 if e > umbral_fijo else 0 for e in ecm]

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
len(y_test)

In [ ]:
classifier_input = Input(shape=(1284,))
classifier_output = Dense(1,activation='sigmoid')(classifier_input)
classifier = Model(classifier_input, classifier_output)

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

In [ ]:
history_encoder = classifier.fit(features_train,y_train, epochs=50, batch_size=128, validation_data = (features_val,y_val))

In [ ]:
plt.plot(history_encoder.history['accuracy'],'ro-', label='accuracy')
plt.plot(history_encoder.history['val_accuracy'],'bo-', label='val_accuracy')
plt.xlabel('Epochs')
plt.legend()

### Intento 2

In [ ]:
model = Sequential([
    #Encoder
    Conv1D(filters=64, kernel_size=5, activation='leaky_relu',padding="same", input_shape=(1284,1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=256, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=512, kernel_size=5, activation='leaky_relu',padding="same"),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(512, activation='leaky_relu'),
    Dense(256, activation='leaky_relu'),
    Dense(128, activation = 'leaky_relu'),
    # Cuello de botella
    Dense(1, activation='sigmoid'),
    #Decoder
    Dense(128, activation='leaky_relu'),
    Dense(256, activation='leaky_relu'),
    Dense(512, activation='leaky_relu'),
    Dense(40960, activation='leaky_relu'), 
    Reshape((80, 256 * 2)),  
    Conv1DTranspose(512, kernel_size=5, activation='leaky_relu', padding='same'),
    UpSampling1D(size=2),
    Conv1DTranspose(256, kernel_size=5, activation='leaky_relu', padding='same'),
    UpSampling1D(size=2),
    Conv1DTranspose(128, kernel_size=2, activation='leaky_relu'),
    UpSampling1D(size=2),
    Conv1DTranspose(64, kernel_size=5, activation='leaky_relu', padding='same'),
    UpSampling1D(size=2),
    Conv1DTranspose(1, kernel_size=5, activation='sigmoid', padding='same')
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,
              loss='mse')
model.summary()

In [ ]:
history = model.fit(X_train, X_train, 
                    epochs=20, 
                    validation_data = (X_val,X_val),
                    batch_size=256)

In [ ]:
plt.plot(history.history['loss'][1:],'bo-')
plt.plot(history.history['val_loss'][-20:],'ro-')
plt.grid()
plt.show()